In [1]:
import argparse
import os
from argparse import RawTextHelpFormatter

from tqdm import tqdm

from TTS.config import load_config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager


In [3]:
c_dataset = load_config("config.json")

In [5]:
meta_data_train, meta_data_eval = load_tts_samples(c_dataset.datasets, eval_split=True)
wav_files = meta_data_train + meta_data_eval

 | > Found 8812 files in /ocean/projects/cie180002p/dagarwal/datasets/kokoro


In [7]:

encoder_manager = SpeakerManager(
    encoder_model_path="model_se.pth.tar",
    encoder_config_path="config_se.json",
    d_vectors_file_path="speakers.json",
    use_cuda=True,
)

class_name_key = encoder_manager.encoder_config.class_name_key

# compute speaker embeddings
speaker_mapping = {}
for idx, wav_file in enumerate(tqdm(wav_files)):
    if isinstance(wav_file, dict):
        class_name = wav_file[class_name_key]
        wav_file = wav_file["audio_file"]
    else:
        class_name = None

    wav_file_name = os.path.basename(wav_file)
    if True and wav_file_name in encoder_manager.clip_ids:
        # get the embedding from the old file
        embedd = encoder_manager.get_embedding_by_clip(wav_file_name)
    else:
        # extract the embedding
        embedd = encoder_manager.compute_embedding_from_clip(wav_file)

    # create speaker_mapping if target dataset is defined
    speaker_mapping[wav_file_name] = {}
    speaker_mapping[wav_file_name]["name"] = class_name
    speaker_mapping[wav_file_name]["embedding"] = embedd




  0%|          | 0/8812 [00:00<?, ?it/s]

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.97
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:False
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:True
 | > db_level:-27.0
 | > stats_path:None
 | > base:10
 | > hop_length:160
 | > win_length:400


100%|██████████| 8812/8812 [19:43<00:00,  7.44it/s] 


In [12]:
if speaker_mapping:
    # save speaker_mapping if target dataset is defined
    if ".json" not in "embeddings/d_vector_file.json":
        mapping_file_path = os.path.join("embeddings/d_vector_file.json", "speakers.json")
    else:
        mapping_file_path = "embeddings/d_vector_file.json"

In [13]:
mapping_file_path

'embeddings/d_vector_file.json'

In [14]:
os.makedirs(os.path.dirname(mapping_file_path), exist_ok=True)

# pylint: disable=W0212
encoder_manager._save_json(mapping_file_path, speaker_mapping)
print("Speaker embeddings saved at:", mapping_file_path)

Speaker embeddings saved at: embeddings/d_vector_file.json


In [11]:
os.makedirs(os.path.dirname(mapping_file_path), exist_ok=True)


FileNotFoundError: [Errno 2] No such file or directory: ''